In [1]:
import tensorflow as tf
physicalDevices = tf.config.list_physical_devices('GPU') 
if physicalDevices:
    tf.config.experimental.set_memory_growth(physicalDevices[0], True)  
from tensorflow import keras
from tensorflow.keras import layers
from data_preprocessor import DataGenerator, GESTURES

import datetime

In [2]:
batch_size=64
epochs=30
input_shape = (20,21,3)
classes = len(GESTURES)
trainGen = DataGenerator('gesture_dataset.h5')

In [3]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.3))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(64)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 63)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 64)            0     

In [4]:
model.fit(generator=trainGen,epochs=epochs)

Epoch 1/30
C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
108/108 [==============================] - 4s 7ms/step - loss: 1.9530 - accuracy: 0.2699
Epoch 2/30
108/108 [==============================] - 1s 7ms/step - loss: 0.6126 - accuracy: 0.7780
Epoch 3/30
108/108 [==============================] - 1s 7ms/step - loss: 0.2738 - accuracy: 0.9198
Epoch 4/30
108/108 [==============================] - 1s 8ms/step - loss: 0.1704 - accuracy: 0.9569
Epoch 5/30
108/108 [==============================] - 1s 7ms/step - loss: 0.0889 - accuracy: 0.9768
Epoch 6/30
108/108 [==============================] - 1s 8ms/step - loss: 0.0652 - accuracy: 0.9828
Epoch 7/30
108/108 [==============================] - 1s 7ms/step - loss: 0.0563 - accuracy: 0.9861
Epoch 8/30
108/108 [=

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tfliteModel = converter.convert()
modelPath = "saved_models/MODEL-{}.tflite".format(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
with open(modelPath, 'wb') as f:
  f.write(tfliteModel)

INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmp6xo2lbeg\assets
INFO:tensorflow:Assets written to: C:\Users\17ami\AppData\Local\Temp\tmp6xo2lbeg\assets
